In [7]:
import pandas as pd

In [8]:
df= pd.read_excel("mapaPatricion.xlsx", skiprows=1)

df.head()


,b. NOMBRE DEL CENTRO DE FAENAMIENTO,a. X,b. Y,c. Z
0,CENTRO DE FAENAMIENTO EMPRESA MUNICIPAL DE SER...,724495.0,9683113.0,2538
1,CENTRO DE FAENAMIENTO DEL GAD MUNICIPAL DE GUA...,747612.0,9680341.0,2228
2,CENTRO DE FAENAMIENTO DEL GAD MUNICIPAL DEL SI...,721314.0,9662528.0,2788
3,CENTRO DE FAENAMIENTO DEL GAD MUNICIPAL DE PAUTE,750021.0,9693269.0,2173
4,CENTRO DE FAENAMIENTO DEL GAD MUNICIPAL PUCARÁ,670274.0,9643909.0,3100


In [9]:
df.columns

Index(['b. NOMBRE DEL CENTRO DE FAENAMIENTO', 'a. X', 'b. Y', 'c. Z'], dtype='object')

In [10]:
import pandas as pd
from pyproj import Transformer

def infer_zona_utm(x):
    if 166000 <= x <= 833000:
        return 17
    elif 833000 < x <= 1660000:  # No aplica a Ecuador
        return 18
    else:
        return 17  # Valor por defecto

def utm_to_latlon_auto(row):
    zona = infer_zona_utm(row['a. X'])
    epsg_utm = f"EPSG:327{zona}"
    transformer = Transformer.from_crs(epsg_utm, "EPSG:4326", always_xy=True)
    lon, lat = transformer.transform(row['a. X'], row['b. Y'])
    return pd.Series({'LATITUD': lat, 'LONGITUD': lon, 'ZONA_UTM': zona})

df[['LATITUD', 'LONGITUD', 'ZONA_UTM']] = df.apply(utm_to_latlon_auto, axis=1)


In [11]:
import folium
import pandas as pd

# Example dataframe


# Create a Folium map centered around Ecuador
m = folium.Map(location=[-2.8, -78.9], zoom_start=8)

# Add markers
for _, row in df.iterrows():
    folium.Marker(
        location=[row['LATITUD'], row['LONGITUD']],
        popup=f"{row['b. NOMBRE DEL CENTRO DE FAENAMIENTO']}<br>Altitud: {row['LONGITUD']} m",
        icon=folium.Icon()
    ).add_to(m)

# Save map
map_path = "mapa_faenamiento.html"
m.save(map_path)
m
